In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5
from DataInfo import ISU_list_MBRN_sum_top,MBRN_sum_top

In [2]:
ISU_list=ISU_list_MBRN_sum_top[:33]
ISU_list=[ISU_CD for ISU_CD in ISU_list if ISU_CD not in ['KR7035720002','KR7008800005', 'KR7014940001']]
Dataname='_10_0823Train'
groupmin=10

In [3]:
len(ISU_list)

30

In [4]:
# TickDataname='GDF2022_5_4_8_25.csv'
ISU_CD=ISU_list[0]
TickDataname="GDF_tick"+"_"+ISU_CD+'_'+str(groupmin)+'.csv'
Data=pd.read_csv('/Data/LOBData/CollectedTickData/ISUTickData/'+TickDataname)
TickGDF=Data

for ISU_CD in ISU_list[1:]:
    TickDataname="GDF_tick"+"_"+ISU_CD+'_'+str(groupmin)+'.csv'
    Data=pd.read_csv('/Data/LOBData/CollectedTickData/ISUTickData/'+TickDataname)
    TickGDF=pd.concat([TickGDF,Data],axis=0)

In [5]:
TGDF=TickGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])
new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()

new_df=new_df.reset_index()
new_df=new_df[(new_df['TM_GP']>=0)&(new_df['TM_GP']<39)]
new_df=new_df.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(new_df.index.levels)
new_df=new_df.reset_index()

Train_df=new_df
Train_df.loc[new_df['TM_GP']==0.,'시가']=Train_df.loc[new_df['TM_GP']==1.,'시가'].values

In [6]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [7]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']
minmaxnormlist=[]
minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격']

In [8]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [9]:
Train_df[loglist]=np.log(Train_df[loglist])

In [10]:
Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()-Train_df[minmaxnormlist].min())

In [11]:
# ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickDataname='GDF_ORD2022_8_1_17_36.csv'
ORDTickGDF=pd.read_csv('/Data/LOBData/CollectedTickData/'+ORDTickDataname)

In [12]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]
OTGDF=OTGDF[OTGDF["ISU_CD"].isin(ISU_list)]

# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)#'norm6feat
# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*OTGDF['ORD_PRC']#'norm6featNewPrc'
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])


In [13]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [14]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

In [15]:
feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [16]:
mbrnlist = MBRN_sum_top[:20]

In [17]:
TickGDF[(TickGDF['ISU_CD']=='KR7035720002')& (TickGDF['TM_GP']>=0.0)][['ORD_DD','TM_GP','시가']]

,ORD_DD,TM_GP,시가


In [18]:
if Train_df['직전체결가격'].isnull().any():
    raise ValueError

In [19]:
A=Train_df[Train_df['직전체결가격'].isnull()]

In [20]:
set(A['ISU_CD'])

set()

In [22]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    
    SaveDirPath='/Data/LOBData/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix

    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
    

5 194
31627832.0
path: /Data/LOBData/TrainData/Train_ORD_5_194_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__5_194_10_0823Train
12 100


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


-4929951.0
path: /Data/LOBData/TrainData/Train_ORD_12_100_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__12_100_10_0823Train
44 1
-351407.0
path: /Data/LOBData/TrainData/Train_ORD_44_1_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__44_1_10_0823Train
17 29
-11815836.0
path: /Data/LOBData/TrainData/Train_ORD_17_29_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__17_29_10_0823Train
2 83
-13381752.0
path: /Data/LOBData/TrainData/Train_ORD_2_83_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__2_83_10_0823Train
4 9997
6120925.0
path: /Data/LOBData/TrainData/Train_ORD_4_9997_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__4_9997_10_0823Train
42 1
2611799.0
path: /Data/LOBData/TrainData/Train_ORD_42_1_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__42_1_10_0823Train
50 91
2069667.0
path: /Data/LOBData/TrainData/Train_ORD_50_91_10_0823Train
path: /Data/LOBData/TrainData/Train_ORD_Label__50_91_10_0823Train
56 17990
-2318482.

In [23]:
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')